In [1]:
import nltk
import random
import pandas as pd
pd.set_option("display.max_rows",500)
import nltk.corpus
random.seed(42)
import numpy as np
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
from nltk.corpus import PlaintextCorpusReader

#warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\x\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


https://www.huffpost.com/entry/obama-is-americas-3rd-gre_b_813868

Can we use text mining to predict the speeches of the top Presidential Orators since 1933?
What are their commonalities?

In [2]:
stop_words = set(stopwords.words('english'))

In [ ]:
corpus = pd.read_csv('data/corpus.csv')

In [ ]:
corpus.head()

In [ ]:
speeches = pd.read_csv('data/presidential_speeches.csv')

In [ ]:
speeches.head()

Import text from R script

In [3]:
ob=open('data/obama1.txt','rU')
raw_o=ob.read()
tokenizer = nltk.RegexpTokenizer(r"\w+")
obama = tokenizer.tokenize(raw_o)

C:\ProgramData\Anaconda3\envs\data620_web\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 'U' mode is deprecated
  """Entry point for launching an IPython kernel.


In [5]:
wa=open('data/washington.txt','rU')
raw_wa=wa.read()
tokenizer = nltk.RegexpTokenizer(r"\w+")
washington = tokenizer.tokenize(raw_wa)

C:\ProgramData\Anaconda3\envs\data620_web\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 'U' mode is deprecated
  """Entry point for launching an IPython kernel.


Update the text, remove stop words, lower case

In [6]:
obama = [word.lower() for word in obama if word.isalpha()]
washington = [word.lower() for word in washington if word.isalpha()]

In [7]:
tokens_without_sw = []
for word in obama:
    if word not in stop_words:
        tokens_without_sw.append(word)

['transcripts',
 'chairman',
 'dean',
 'great',
 'friend',
 'dick',
 'durbin',
 'fellow',
 'citizens',
 'great',
 'nation',
 'profound',
 'gratitude',
 'great',
 'humility',
 'accept',
 'nomination',
 'presidency',
 'united',
 'states',
 'let',
 'express',
 'thanks',
 'historic',
 'slate',
 'candidates',
 'accompanied',
 'journey',
 'especially',
 'one',
 'traveled',
 'farthest',
 'champion',
 'working',
 'americans',
 'inspiration',
 'daughters',
 'hillary',
 'rodham',
 'clinton',
 'president',
 'clinton',
 'last',
 'night',
 'made',
 'case',
 'change',
 'make',
 'ted',
 'kennedy',
 'embodies',
 'spirit',
 'service',
 'next',
 'vice',
 'president',
 'united',
 'states',
 'joe',
 'biden',
 'thank',
 'grateful',
 'finish',
 'journey',
 'one',
 'finest',
 'statesmen',
 'time',
 'man',
 'ease',
 'everyone',
 'world',
 'leaders',
 'conductors',
 'amtrak',
 'train',
 'still',
 'takes',
 'home',
 'every',
 'night',
 'love',
 'life',
 'next',
 'first',
 'lady',
 'michelle',
 'obama',
 'sasha'

In [ ]:
obama_str = str(obama)
word_tokens = word_tokenize(obama_str)
tokens_without_sw = [w for w in word_tokens if not w in stop_words]
#tokens_without_sw = []        
obama_filtered = ("").join(tokens_without_sw)
obama_filtered = obama_filtered.replace(' , ', '')
obama_filtered = obama_filtered.replace(" ''", "")
obama_filtered = obama_filtered.replace("''", "")

In [9]:
len(obama)

207326

In [10]:
len(obama)/2000

103.663

For ease of segmenting, I divided the length of the text by 2000 to get the amount of segments (1,101,169/2000 = ~550 Segments)

In [ ]:
obama1=[]
for i in range(760):
    obama1.append([obama_filtered[i*2000:(i+1)*2000], 'ob'])

In [ ]:
washington = [word.lower() for word in washington if word.isalpha()]
washington_str = str(washington)
word_tokens = word_tokenize(washington_str)
tokens_without_sw = [w for w in word_tokens if not w in stop_words]
#tokens_without_sw = []        
washington_filtered = ("").join(tokens_without_sw)
washington_filtered = washington_filtered.replace(' , ', '')
washington_filtered = washington_filtered.replace(" ''", "")
washington_filtered = washington_filtered.replace("''", "")


In [ ]:
#reagan_filtered

In [ ]:
len(washington_filtered)/2000

In [ ]:
washington1=[]
for i in range(775):
    washington1.append([washington_filtered[i*2000:(i+1)*2000], 're'])

## Feature Extraction

Creating the list of 1000 most frequent words

In [ ]:
wo = obama+washington
all_words = nltk.FreqDist(w.lower() for w in wo)
word_features = list(all_words)[:1000] #1000 most frequent words
wlist=[]
for i in range(0,1000,50):
    df=pd.DataFrame(word_features[i:(i+50)])
    df.columns=['50 words']
    wlist.append(df)   
pd.concat(wlist, axis=1)

From text book

In [ ]:
def document_features(document): 
    document_words = set(document) 
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features   

run features on Washington's speeches 

In [ ]:
features = document_features(washington)
list(features.items())[10:30]

## Test/Train Dataset

In [ ]:
speeches = obama1+washington1

In [ ]:
import random
random.shuffle(speeches)
featuresets = [(document_features(d), c) for (d,c) in speeches]
len(featuresets)

In [ ]:
train_set, test_set = featuresets[:1000], featuresets[1000:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
print(nltk.classify.accuracy(classifier, test_set))

92%, the concept works!

Does it get better or worse if we focus on speakers all from the same general era?.

### Adding Roosevelt

In [ ]:
ro=open('data/roosevelt.txt','rU')
raw_r=ro.read()
tokens_r = nltk.word_tokenize(raw_r)
roosevelt = nltk.Text(tokens_r)
roosevelt = [word.lower() for word in roosevelt if word.isalpha()]
roosevelt_str = str(roosevelt)
word_tokens = word_tokenize(roosevelt_str)
tokens_without_sw = [w for w in word_tokens if not w in stop_words]
#tokens_without_sw = []        
roosevelt_filtered = (" ").join(tokens_without_sw)
roosevelt_filtered = roosevelt_filtered.replace(' , ', '')
roosevelt_filtered = roosevelt_filtered.replace(" ''", " ")

len(roosevelt_filtered)/2000

In [ ]:
roosevelt1=[]
for i in range(365):
    roosevelt1.append([roosevelt_filtered[i*2000:(i+1)*2000], 'ro'])

In [ ]:
ro = obama+roosevelt
all_words = nltk.FreqDist(w.lower() for w in ro)
word_features = list(all_words)[:200] #200 most frequent words

speeches = obama1+roosevelt1

In [ ]:
random.shuffle(speeches)
featuresets = [(document_features(d), c) for (d,c) in speeches]
len(featuresets)

In [ ]:
train_set, test_set = featuresets[:100], featuresets[100:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set)) 

## Kennedy

In [ ]:
ke=open('data/kennedy.txt','rU')
raw_k=ke.read()
tokens_k = nltk.word_tokenize(raw_k)
kennedy = nltk.Text(tokens_k)
kennedy = [word.lower() for word in kennedy if word.isalpha()]
kennedy_str = str(kennedy)
word_tokens = word_tokenize(kennedy_str)
tokens_without_sw = [w for w in word_tokens if not w in stop_words]
#tokens_without_sw = []        
kennedy_filtered = (" ").join(tokens_without_sw)
kennedy_filtered = kennedy_filtered.replace(' , ', '')
kennedy_filtered = kennedy_filtered.replace(" ''", " ")

len(kennedy_filtered)/2000

In [ ]:
kennedy1=[]
for i in range(336):
    kennedy1.append([kennedy_filtered[i*2000:(i+1)*2000], 'ke'])

In [ ]:
ork = obama+roosevelt+kennedy
all_words = nltk.FreqDist(w.lower() for w in ork)
word_features = list(all_words)[:200] #200 most frequent words

speeches = obama1+roosevelt1+kennedy1

In [ ]:
random.shuffle(speeches)
featuresets = [(document_features(d), c) for (d,c) in speeches]
len(featuresets)

In [ ]:
train_set, test_set = featuresets[:100], featuresets[100:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set)) 

words words

## Clinton

In [ ]:
cl=open('data/clinton.txt','rU')
raw_c=cl.read()
tokens_c = nltk.word_tokenize(raw_c)
clinton = nltk.Text(tokens_c)
clinton = [word.lower() for word in clinton if word.isalpha()]
clinton_str = str(clinton)
word_tokens = word_tokenize(clinton_str)
tokens_without_sw = [w for w in word_tokens if not w in stop_words]
#tokens_without_sw = []        
clinton_filtered = (" ").join(tokens_without_sw)
clinton_filtered = clinton_filtered.replace(' , ', '')
clinton_filtered = clinton_filtered.replace(" ''", " ")

len(clinton_filtered)/2000

In [ ]:
clinton1=[]
for i in range(367):
    clinton1.append([clinton_filtered[i*2000:(i+1)*2000], 'cl'])

In [ ]:
orkc = obama+roosevelt+kennedy+clinton
all_words = nltk.FreqDist(w.lower() for w in orkc)
word_features = list(all_words)[:200] #200 most frequent words

speeches = obama1+roosevelt1+kennedy1+clinton1

In [ ]:
random.shuffle(speeches)
featuresets = [(document_features(d), c) for (d,c) in speeches]
len(featuresets)

In [ ]:
train_set, test_set = featuresets[:100], featuresets[100:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set)) 

words words

## Reagan

In [ ]:
re=open('data/reagan.txt','rU')
raw_re=re.read()
tokens_re = nltk.word_tokenize(raw_re)
reagan = nltk.Text(tokens_re)
reagan = [word.lower() for word in reagan if word.isalpha()]
reagan_str = str(reagan)
word_tokens = word_tokenize(reagan_str)
tokens_without_sw = [w for w in word_tokens if not w in stop_words]
#tokens_without_sw = []        
reagan_filtered = (" ").join(tokens_without_sw)
reagan_filtered = reagan_filtered.replace(' , ', '')
reagan_filtered = reagan_filtered.replace(" ''", " ")

len(reagan_filtered)/2000

In [ ]:
reagan1=[]
for i in range(564):
    reagan1.append([reagan_filtered[i*2000:(i+1)*2000], 'cl'])

In [ ]:
orkcr = obama+roosevelt+kennedy+clinton+reagan
all_words = nltk.FreqDist(w.lower() for w in orkcr)
word_features = list(all_words)[:200] #200 most frequent words

speeches = obama1+roosevelt1+kennedy1+clinton1+reagan1

In [ ]:
random.shuffle(speeches)
featuresets = [(document_features(d), c) for (d,c) in speeches]
len(featuresets)

In [ ]:
train_set, test_set = featuresets[:100], featuresets[100:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set)) 

38% proves our initial concern.  Since we are basing this research on speeches of people all from near the same era, it can be extremely difficult to teach a model to predict as more and more text gets added.  